In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import pyodbc
from lmfit.models import PolynomialModel
import math

In [ ]:
sql_conn = pyodbc.connect('DRIVER={SQL Server};'
                            'SERVER=L_AAGDATIX;'
                            'DATABASE=DatixCRM;'
                            'Trusted_Connection=yes') 
query = "set transaction isolation level read uncommitted select inc_dincident,inc_dreported from DatixCRM.dbo.incidents_main where inc_type='PAT' and inc_category='MEDIC' and inc_organisation='QA'"
df = pd.read_sql(query, sql_conn)
print(df)

In [ ]:
df_inc = df.copy()
df_inc = df_inc[df_inc.inc_dincident > '2016-03-31']
df_inc = df_inc[df_inc.inc_dincident < '2019-10-01']
print(df_inc)

In [ ]:
df_inc['month'] = df_inc['inc_dincident'].dt.month
print(df_inc)

In [ ]:
month_occurences = np.array([3,3,3,4,4,4,4,4,4,3,3,3])
print(month_occurences)

In [ ]:
days_in_month = np.ones(12)*31
days_in_month[1] = 28
days_in_month[3] = 30
days_in_month[5] = 30
days_in_month[8] = 30
days_in_month[10] = 30
print(days_in_month)

In [ ]:
division_factor = month_occurences*days_in_month
print(division_factor)

In [ ]:
bins = [i+0.5 for i in range(13)]
data_x = [i+1 for i in range(12)]
data_y_tot,_ = np.histogram(df_inc.month.values, bins=bins)
data_y = data_y_tot/division_factor
data_y_err = np.sqrt(data_y_tot)/division_factor
plt.bar(data_x,data_y,label='Data')
plt.bar(data_x,data_y_err,bottom=data_y-data_y_err/2,alpha=0.5,color='none',hatch="////",label='Uncertainty')
#plt.legend(frameon=False)
ax = plt.gca()
locs, labels = plt.xticks()
plt.xticks(data_x,labels=['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec'])
plt.xlabel('Month');
plt.ylabel('# events per day')
plt.title('Daily number of patient medicine safety events at QA')
img = plt.imread("nhsportsmouth.png");
ax = plt.axes([0.125+0.01,0.65, 0.3, 0.3], frameon=True)  # Change the numbers in this array to position your image [left, bottom, width, height])
#ax.imshow(img)
ax.axis('off')  # get rid of the ticks and ticklabels

In [ ]:
k = 0
reduced_chi_sq = 3
while reduced_chi_sq > 2 or reduced_chi_sq < 0.5:
    # data fit
    polynomial_mod = PolynomialModel(k)
    pars = polynomial_mod.guess(data_y, x=data_x, c0=data_y[0])
    model = polynomial_mod
    out = model.fit(data_y, pars, x=data_x, weights=1/data_y_err)

    plt.bar(data_x,data_y,label='Data')
    plt.bar(data_x,data_y_err,bottom=data_y-data_y_err/2,alpha=0.5,color='none',hatch="////",label='Uncertainty')
    plt.plot(data_x, out.best_fit, '-r', label='Fit '+r'$\chi^2/N_{df}$ = '+str(round(out.redchi,1)))
    plt.legend(loc=(1.04,0))
    ax = plt.gca()
    locs, labels = plt.xticks()
    plt.xticks(data_x,labels=['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec'])
    plt.xlabel('Month');
    plt.ylabel('# events per day')
    plt.title('Daily number of patient medicine safety events at QA')
    img = plt.imread("nhsportsmouth.png");
    ax = plt.axes([0.91,0.65, 0.3, 0.3], frameon=True)  # Change the numbers in this array to position your image [left, bottom, width, height])
    ax.imshow(img)
    ax.axis('off')  # get rid of the ticks and ticklabels
    
    reduced_chi_sq = out.redchi
    k+=1

In [ ]:
df_rep = df.copy()
df_rep = df_rep[df_rep.inc_dreported > '2016-03-31']
df_rep = df_rep[df_rep.inc_dreported < '2019-10-01']

In [ ]:
df_rep['month'] = df_rep['inc_dreported'].dt.month

In [ ]:
k = 0
reduced_chi_sq = 3
while reduced_chi_sq > 2 or reduced_chi_sq < 0.5:
    # data fit
    polynomial_mod = PolynomialModel(k)
    pars = polynomial_mod.guess(data_y, x=data_x, c0=data_y[0])
    model = polynomial_mod
    out = model.fit(data_y, pars, x=data_x, weights=1/data_y_err)

    plt.bar(data_x,data_y,label='Data')
    plt.bar(data_x,data_y_err,bottom=data_y-data_y_err/2,alpha=0.5,color='none',hatch="////",label='Uncertainty')
    plt.plot(data_x, out.best_fit, '-r', label='Fit '+r'$\chi^2/N_{df}$ = '+str(round(out.redchi,1)))
    plt.legend(loc=(1.04,0))
    ax = plt.gca()
    locs, labels = plt.xticks()
    plt.xticks(data_x,labels=['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec'])
    plt.xlabel('Month');
    plt.ylabel('# events per day')
    plt.title('Daily number of patient medicine safety events at QA')
    img = plt.imread("nhsportsmouth.png");
    ax = plt.axes([0.91,0.65, 0.3, 0.3], frameon=True)  # Change the numbers in this array to position your image [left, bottom, width, height])
    ax.imshow(img)
    ax.axis('off')  # get rid of the ticks and ticklabels
    
    reduced_chi_sq = out.redchi
    k+=1

In [ ]:
# Import figure from bokeh.plotting
from bokeh.plotting import figure

# Import output_file and show from bokeh.io
from bokeh.io import show, output_notebook

from bokeh.models import ColumnDataSource, Band, Legend, LegendItem, Span, HoverTool


groups= ['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']
counts = data_y
error = data_y_err
upper = [x+e for x,e in zip(counts, error) ]
lower = [x-e for x,e in zip(counts, error) ]
color = ['blue','blue','green','green','green','red','red','red','yellow','yellow','yellow','blue']

source = ColumnDataSource(data=dict(groups=groups, counts=counts, upper=upper, lower=lower, color=color))

p = figure(x_range=groups, plot_height=450, title="Daily medicine patient safety events at QA", y_range=(0,math.ceil(max(counts))-0.2),
          x_axis_label='Month',y_axis_label='events per day',
          tools='pan,box_select,box_zoom,wheel_zoom,save,reset,help')

p.vbar(x='groups', top='counts', width=0.9, source=source, 
       line_color='white', color='color',nonselection_alpha=0, 
         hover_fill_color='black')
r_err = p.vbar(x='groups', bottom='lower', top='upper', width=0.9, source=source, fill_color='grey', fill_alpha=0.9,
      line_color='white')
r_linear = p.line(groups,out.best_fit,line_color='purple')

p.xgrid.grid_line_color = None

legend = Legend(items=[
    LegendItem(label="winter", renderers=[p.vbar(x=0, top=0, width=0.9, color='blue')]),
    LegendItem(label="spring", renderers=[p.vbar(x=0, top=0, width=0.9, color='green')]),
    LegendItem(label="summer", renderers=[p.vbar(x=0, top=0, width=0.9, color='red')]),
    LegendItem(label="autumn", renderers=[p.vbar(x=0, top=0, width=0.9, color='yellow')]),
    LegendItem(label="uncertainty", renderers=[r_err]),
    LegendItem(label='fit χ²/Nᵈᶠ = '+str(round(out.redchi,1)), renderers=[r_linear]),
])
p.add_layout(legend)
p.legend.location = "bottom_right"

# Create a HoverTool: hover
hover = HoverTool(tooltips=None,mode='hline')

# Add the hover tool to the figure p
p.add_tools(hover)

output_notebook()


In [ ]:
show(p)